In [1]:
'''Import packages and set options'''
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
import pickle

pd.set_option('display.max_columns', None)

In [2]:
'''Declare city and state variables'''
city = 'Wilmington'
state = 'NC'
search_string = city + ', ' + state

In [3]:
'''Get API Keys'''
api_keys = pd.read_csv('api_key.csv')
rapid_api_key = api_keys.loc[api_keys['API'] == 'rapid']['KEY'][0]

In [28]:
'''Set search parameters'''
home_type = "Houses"
minPrice = "0"
maxPrice = "400000"
sqftMin = "1000"
status_type = "RecentlySold"
page = "1"

In [29]:
'''Get data from Zillow - update'''
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

#Parameters for houses that are currently for sale
for_sale_querystring = {"location":search_string,
                        "page":page,
                        "home_type":home_type,
                        "minPrice":minPrice,
                        "maxPrice":maxPrice,
                        "sqftMin":sqftMin}
#Parameters for houses that were recently sold
sold_querystring = {"location":search_string,
                    "page":page,
                    "status_type":status_type,
                    "minPrice":minPrice,
                    "maxPrice":maxPrice,
                    "home_type":home_type,
                    "sqftMin":sqftMin}

headers = {
    "X-RapidAPI-Key": rapid_api_key,
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

In [30]:
'''Retrieve information for houses currently for sale'''
for_sale_response = requests.request("GET", url, headers=headers, params=for_sale_querystring)
for_sale_json = for_sale_response.json()
#print(for_sale_json)
num_page = for_sale_json['totalPages']
if num_page > 1:
    for i in range(1, num_page):
        page = int(page)
        page += 1
        for_sale_querystring.update({"page":str(page)})
        temp_response = requests.request("GET", url, headers=headers, params=for_sale_querystring)
        temp_json = temp_response.json()
        time.sleep(20)
        #print(temp_json)
        for_sale_json['props'].extend(temp_json['props'])

In [ ]:
'''Retrieve information for houses which were recently sold'''
sold_response = requests.request("GET", url, headers=headers, params=sold_querystring)
sold_json = sold_response.json()
#print(sold_json)
num_page = int(sold_json['totalResultCount']/40)
time.sleep(15)
print("NumPage:", num_page)
max_price = None

#If there are more than 20 pages of data, the data needs to be broken into smaller chunks
if num_page >= 20:
    #Divide data into smaller chunks by housing price, set initial range and get initial file
    if max_price is None:
        max_price = int(int(maxPrice)/10)
    sold_querystring.update({"page":"1", "maxPrice":str(max_price)})
    sold_response = requests.request("GET", url, headers=headers, params=sold_querystring)
    sold_json = sold_response.json()
    time.sleep(15)

    #Outside loop that incremements selling price range, The while loopsets initial parameters and then increments them
    while max_price <= int(maxPrice):
        sold_querystring.update({"maxPrice":str(max_price), "minPrice": str(minPrice)})
        temp_response = requests.request("GET", url, headers=headers, params=sold_querystring)
        temp_json = temp_response.json()
        print("Total Result Count for min price", minPrice, "and max price", max_price, "is", temp_json['totalResultCount'])
        sub_num_page = temp_json['totalPages']
        sub_num_results = temp_json['totalResultCount']
        print("Number of subpages", sub_num_page, "sub_num_results", sub_num_results)
        time.sleep(20)
        #If this is not the first iteration and there are more than 800 results, 
        #increment min_price by a smaller amount to decrease number of records
        if max_price != int(int(maxPrice)/10):
            if sub_num_results >= 800:
                max_price = round(minPrice, -3) + int(int(maxPrice)/20)
                sold_querystring.update({"maxPrice":str(max_price)})
                
        #inside loop that obtains pages for each price range
        for i in range(1, sub_num_page):
            print("MinPrice", minPrice, "MaxPrice", max_price)
            page = int(page)
            page += 1
            sold_querystring.update({"page":str(page)})
            temp_response = requests.request("GET", url, headers=headers, params=sold_querystring)
            temp_json = temp_response.json()
            time.sleep(20)
            #print(temp_json)
            try:
                sold_json['props'].extend(temp_json['props'])
            except Exception as e:
                print(repr(e))
                pass
            print('There are', len(sold_json['props']), "properties in sold_json after", i, "iterations.")

        #increments price range for outside loop
        minPrice = max_price + 1
        max_price = max_price + int(int(maxPrice)/10)
        page = "1"
        print("min price after increment:", minPrice, "max price after increment:", max_price)

        #obtain initial data for next iteration
        temp_response = requests.request("GET", url, headers=headers, params=sold_querystring)
        #print(temp_response)
        try:
            sub_num_page = temp_response['totalPages']
        except Exception as e:
            print(repr(e))
            pass

elif num_page > 1:
    for i in range(1, num_page):
        page = int(page)
        page += 1
        sold_querystring.update({"page":str(page)})
        temp_response = requests.request("GET", url, headers=headers, params=sold_querystring)
        temp_json = temp_response.json()
        time.sleep(15)
        print("temp_json",temp_json)
        try:
            sold_json['props'].extend(temp_json['props'])
        except Exception as e:
            print(repr(e))
            pass
        print("sold_json",sold_json)
        print('Greater than 1:', len(sold_json['props']))

In [33]:
len(for_sale_json['props'])

86

In [32]:
'''Pickle currently for_sale datafile'''
with open('for_sale.pkl', 'wb') as f:
    pickle.dump(for_sale_json, f)

In [26]:
'''Pickle recently_sold datafile'''
with open('sold.pkl', 'wb') as f:
    pickle.dump(sold_json, f)

In [ ]:
'''Open for_sale pickle file'''
with open('for_sale.pkl', 'rb') as f:
    for_sale_json = pickle.load(f)

In [ ]:
'''Open recently sold pickle file'''
with open('sold.pkl', 'rb') as f:
    sold_json = pickle.load(f)

In [34]:
'''Convert output to dataframe'''
for_sale_df = pd.json_normalize(data = for_sale_json['props'])
sold_df = pd.json_normalize(data = sold_json['props'])

In [35]:
for_sale_df

,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,unit,listingSubType.is_newHome
0,None,SINGLE_FAMILY,0.46,"134 Whipporwill Lane, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/aaea5a8fd22...,375000,3,-77.865320,34.153725,FOR_SALE,54340660,None,-1,2,1644.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
1,None,SINGLE_FAMILY,0.36,"610 Manassas Drive, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/126526881df...,325000,3,-77.895935,34.120950,FOR_SALE,54344070,None,-1,2,1319.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
2,None,SINGLE_FAMILY,0.69,"632 Colony Circle N, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/12025dedd81...,375000,3,-77.881630,34.168335,FOR_SALE,54334330,None,-1,3,1810.0,USA,USD,acres,True,True,True,NaN,NaN,NaN
3,None,SINGLE_FAMILY,0.26,"118 Presidio Drive, Wilmington, NC 28412",https://photos.zillowstatic.com/fp/87c88da407d...,285000,3,-77.914740,34.112270,FOR_SALE,54343476,None,-1,2,1301.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
4,None,SINGLE_FAMILY,0.57,"669 Hidden Valley Road, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/587c523a4e5...,359900,3,-77.891650,34.151524,FOR_SALE,54339572,None,-1,3,1788.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,None,SINGLE_FAMILY,NaN,"ARDEN Plan, Riverside",https://photos.zillowstatic.com/fp/de668de5c9e...,394990,3,-77.927040,34.288140,FOR_SALE,2079470492,None,-1,3,1885.0,USA,USD,None,True,NaN,True,BUILDER_PLAN,# I0D8SI,True
82,None,SINGLE_FAMILY,NaN,"CALI Plan, Riverside",https://photos.zillowstatic.com/fp/1f2273273c5...,388990,4,-77.927040,34.288140,FOR_SALE,2079470489,None,-1,2,1774.0,USA,USD,None,True,NaN,NaN,BUILDER_PLAN,# 7LULNK,True
83,None,SINGLE_FAMILY,NaN,"ARIA Plan, Riverside",https://photos.zillowstatic.com/fp/52f2d1a574e...,372990,3,-77.927040,34.288140,FOR_SALE,2079470486,None,-1,2,1618.0,USA,USD,None,True,NaN,NaN,BUILDER_PLAN,# UD84Q1,True
84,None,SINGLE_FAMILY,NaN,"MACON Plan, Riverside",https://photos.zillowstatic.com/fp/c1be503f8a0...,352990,3,-77.927040,34.288140,FOR_SALE,2079470487,None,-1,2,1343.0,USA,USD,None,True,NaN,NaN,BUILDER_PLAN,# O9X2OJ,True


In [36]:
'''Check difference between two tables'''
difference = list(set(for_sale_df.columns) - set(sold_df.columns))
difference

[]

In [37]:
for_sale_df

,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,unit,listingSubType.is_newHome
0,None,SINGLE_FAMILY,0.46,"134 Whipporwill Lane, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/aaea5a8fd22...,375000,3,-77.865320,34.153725,FOR_SALE,54340660,None,-1,2,1644.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
1,None,SINGLE_FAMILY,0.36,"610 Manassas Drive, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/126526881df...,325000,3,-77.895935,34.120950,FOR_SALE,54344070,None,-1,2,1319.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
2,None,SINGLE_FAMILY,0.69,"632 Colony Circle N, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/12025dedd81...,375000,3,-77.881630,34.168335,FOR_SALE,54334330,None,-1,3,1810.0,USA,USD,acres,True,True,True,NaN,NaN,NaN
3,None,SINGLE_FAMILY,0.26,"118 Presidio Drive, Wilmington, NC 28412",https://photos.zillowstatic.com/fp/87c88da407d...,285000,3,-77.914740,34.112270,FOR_SALE,54343476,None,-1,2,1301.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
4,None,SINGLE_FAMILY,0.57,"669 Hidden Valley Road, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/587c523a4e5...,359900,3,-77.891650,34.151524,FOR_SALE,54339572,None,-1,3,1788.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,None,SINGLE_FAMILY,NaN,"ARDEN Plan, Riverside",https://photos.zillowstatic.com/fp/de668de5c9e...,394990,3,-77.927040,34.288140,FOR_SALE,2079470492,None,-1,3,1885.0,USA,USD,None,True,NaN,True,BUILDER_PLAN,# I0D8SI,True
82,None,SINGLE_FAMILY,NaN,"CALI Plan, Riverside",https://photos.zillowstatic.com/fp/1f2273273c5...,388990,4,-77.927040,34.288140,FOR_SALE,2079470489,None,-1,2,1774.0,USA,USD,None,True,NaN,NaN,BUILDER_PLAN,# 7LULNK,True
83,None,SINGLE_FAMILY,NaN,"ARIA Plan, Riverside",https://photos.zillowstatic.com/fp/52f2d1a574e...,372990,3,-77.927040,34.288140,FOR_SALE,2079470486,None,-1,2,1618.0,USA,USD,None,True,NaN,NaN,BUILDER_PLAN,# UD84Q1,True
84,None,SINGLE_FAMILY,NaN,"MACON Plan, Riverside",https://photos.zillowstatic.com/fp/c1be503f8a0...,352990,3,-77.927040,34.288140,FOR_SALE,2079470487,None,-1,2,1343.0,USA,USD,None,True,NaN,NaN,BUILDER_PLAN,# O9X2OJ,True


In [38]:
'''Drop columns that are not in both frames'''
for_sale_df.drop(difference, axis =1, inplace=True)

In [39]:
'''Concatenate dataframes into frame'''
frames = [for_sale_df, sold_df]
all_sales = pd.concat(frames)
all_sales

,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,unit,listingSubType.is_newHome,listingSubType.is_FSBO,listingSubType.is_accepting_backup_offers
0,None,SINGLE_FAMILY,0.46,"134 Whipporwill Lane, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/aaea5a8fd22...,375000,3.0,-77.865320,34.153725,FOR_SALE,54340660,None,-1,2.0,1644.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN,NaN,NaN
1,None,SINGLE_FAMILY,0.36,"610 Manassas Drive, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/126526881df...,325000,3.0,-77.895935,34.120950,FOR_SALE,54344070,None,-1,2.0,1319.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN,NaN,NaN
2,None,SINGLE_FAMILY,0.69,"632 Colony Circle N, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/12025dedd81...,375000,3.0,-77.881630,34.168335,FOR_SALE,54334330,None,-1,3.0,1810.0,USA,USD,acres,True,True,True,NaN,NaN,NaN,NaN,NaN
3,None,SINGLE_FAMILY,0.26,"118 Presidio Drive, Wilmington, NC 28412",https://photos.zillowstatic.com/fp/87c88da407d...,285000,3.0,-77.914740,34.112270,FOR_SALE,54343476,None,-1,2.0,1301.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN,NaN,NaN
4,None,SINGLE_FAMILY,0.57,"669 Hidden Valley Road, Wilmington, NC 28409",https://photos.zillowstatic.com/fp/587c523a4e5...,359900,3.0,-77.891650,34.151524,FOR_SALE,54339572,None,-1,3.0,1788.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8792,1657782000000,SINGLE_FAMILY,9147.60,"6624 Wedderburn Drive, Wilmington, NC 28412",https://photos.zillowstatic.com/fp/7e42f1f99ec...,379000,4.0,-77.906310,34.094284,RECENTLY_SOLD,54345915,None,-1,3.0,1880.0,USA,USD,sqft,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8793,1657695600000,SINGLE_FAMILY,0.30,"4913 Pleasant Oaks Drive, Wilmington, NC 28412",https://photos.zillowstatic.com/fp/2f2b1af4ad3...,375000,3.0,-77.917830,34.143887,RECENTLY_SOLD,54340946,None,-1,3.0,1749.0,USA,USD,acres,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8794,1657609200000,SINGLE_FAMILY,0.30,"1005 Shadow Moss Court, Wilmington, NC 28412",https://photos.zillowstatic.com/fp/7c3eb2b0448...,365000,3.0,-77.919370,34.152992,RECENTLY_SOLD,54336597,None,-1,2.0,1341.0,USA,USD,acres,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8795,1657522800000,SINGLE_FAMILY,0.75,"629 Creekwood Road, Wilmington, NC 28411",https://photos.zillowstatic.com/fp/defecb780c8...,380000,4.0,-77.762150,34.304825,RECENTLY_SOLD,54283681,None,-1,2.0,1390.0,USA,USD,acres,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
'''Check datatypes'''
all_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8883 entries, 0 to 8796
Data columns (total 26 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   dateSold                                   8797 non-null   object 
 1   propertyType                               8883 non-null   object 
 2   lotAreaValue                               8784 non-null   float64
 3   address                                    8883 non-null   object 
 4   imgSrc                                     8883 non-null   object 
 5   price                                      8883 non-null   int64  
 6   bedrooms                                   8828 non-null   float64
 7   longitude                                  8522 non-null   float64
 8   latitude                                   8522 non-null   float64
 9   listingStatus                              8883 non-null   object 
 10  zpid                    

In [41]:
'''Convert dateSold to datetime'''
all_sales['dateSold']= pd.to_datetime(all_sales['dateSold'])
all_sales['dateSold'] = pd.to_datetime(all_sales['dateSold'].dt.strftime('%Y-%m'))

In [42]:
'''Coordinate box of desired living area'''
UL = 34.250170, -77.946410
UR = 34.243651, -77.915227
LL = 34.225592, -77.944480
LR = 34.226329, -77.911553

print(UL[1])

-77.94641


In [43]:
'''Define desired geographic area'''
relevant_homes = all_sales.query("longitude >= @UL[1] and longitude <= @LR[1] and latitude <= @UL[0] and latitude >= @LR[0]")
relevant_homes

,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,unit,listingSubType.is_newHome,listingSubType.is_FSBO,listingSubType.is_accepting_backup_offers
13,NaT,SINGLE_FAMILY,2178.000000,"917 Chestnut Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/4560b29811f...,289000,3.0,-77.937355,34.238650,FOR_SALE,54301824,None,-1,2.0,1151.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,NaN,NaN
27,NaT,SINGLE_FAMILY,2613.600000,"707 N 11th Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/757e38dfb46...,275000,2.0,-77.935905,34.244090,FOR_SALE,54300701,None,-1,2.0,1149.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,NaN,NaN
33,NaT,SINGLE_FAMILY,4791.600000,"105 N 13th Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/0b515c83515...,239900,3.0,-77.933334,34.238210,FOR_SALE,54301987,None,-1,2.0,1463.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,NaN,NaN
34,NaT,SINGLE_FAMILY,0.250000,"449 N 21st Street, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/608ccf33669...,315000,3.0,-77.924840,34.244804,FOR_SALE,54300922,None,-1,2.0,1366.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN,NaN,NaN
40,NaT,SINGLE_FAMILY,1742.400000,"319 Anderson Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/1f63ff91311...,149997,2.0,-77.938890,34.240288,FOR_SALE,54300588,None,-1,1.0,1038.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635,1970-01-01,SINGLE_FAMILY,5662.000000,"605 S 3rd St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,375000,3.0,-77.944670,34.227585,RECENTLY_SOLD,54310682,None,-1,2.0,2132.0,USA,USD,sqft,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8637,1970-01-01,SINGLE_FAMILY,3484.000000,"219 N 7th St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,380000,3.0,-77.941330,34.239124,RECENTLY_SOLD,54301614,None,-1,2.0,1821.0,USA,USD,sqft,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8651,1970-01-01,SINGLE_FAMILY,0.429982,"418 Forest Hills Dr, Wilmington, NC 28403",https://maps.googleapis.com/maps/api/staticmap...,373500,4.0,-77.913490,34.232530,RECENTLY_SOLD,54310438,None,-1,3.0,2098.0,USA,USD,acres,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8653,1970-01-01,SINGLE_FAMILY,0.419995,"209 S 15th St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,380000,2.0,-77.929510,34.233814,RECENTLY_SOLD,54309388,None,-1,2.0,1568.0,USA,USD,acres,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
'''Save file as csv'''
relevant_homes.to_csv('relevant_homes.csv')
relevant_homes = pd.read_csv('relevant_homes.csv')

In [45]:
'''Calculate Price per sqft'''
relevant_homes.loc[relevant_homes['lotAreaUnit'] == 'sqft', 'Price/sqft'] = relevant_homes['price']/relevant_homes['livingArea']
relevant_homes.sort_values('Price/sqft')

,Unnamed: 0,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,unit,listingSubType.is_newHome,listingSubType.is_FSBO,listingSubType.is_accepting_backup_offers,Price/sqft
31,72,1970-01-01,SINGLE_FAMILY,4800.000000,"1904 Church St, Wilmington, NC 28403",https://maps.googleapis.com/maps/api/staticmap...,500,3.0,-77.924220,34.230730,RECENTLY_SOLD,54310182,NaN,-1,1.0,1144.0,USA,USD,sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.437063
23,32,1970-01-01,SINGLE_FAMILY,4800.000000,"1904 Church St, Wilmington, NC 28403",https://maps.googleapis.com/maps/api/staticmap...,500,3.0,-77.924220,34.230730,RECENTLY_SOLD,54310182,NaN,-1,1.0,1144.0,USA,USD,sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.437063
887,8169,1970-01-01,SINGLE_FAMILY,0.120000,"709 S 4th St,",https://photos.zillowstatic.com/fp/c2d66e68991...,1850,4.0,-77.943214,34.226383,RECENTLY_SOLD,54310784,NaN,-1,2.0,1960.0,USA,USD,sqft,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.943878
75,249,1970-01-01,SINGLE_FAMILY,2909.000000,"919 Grace St, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/a90528fe340...,1800,3.0,-77.937710,34.239735,RECENTLY_SOLD,54301526,NaN,-1,1.5,1000.0,USA,USD,sqft,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.800000
66,232,1970-01-01,SINGLE_FAMILY,3484.000000,"1802 Colwell Ave, Wilmington, NC 28403",https://photos.zillowstatic.com/fp/b2f2bc3cd02...,2300,3.0,-77.925240,34.228924,RECENTLY_SOLD,54310222,NaN,-1,2.0,1038.0,USA,USD,sqft,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.215800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,8048,1970-01-01,SINGLE_FAMILY,0.300000,"208 N 27th Street, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/dfc0e342be3...,359900,3.0,-77.911705,34.242004,RECENTLY_SOLD,54301212,NaN,-1,2.0,1472.0,USA,USD,acres,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886,8146,1970-01-01,SINGLE_FAMILY,0.700000,"1502 Dock Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/e3f1ba370ca...,379900,3.0,-77.929910,34.235497,RECENTLY_SOLD,54302163,NaN,-1,2.0,1505.0,USA,USD,acres,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909,8606,1970-01-01,SINGLE_FAMILY,0.679982,"2310 Metts Ave, Wilmington, NC 28403",https://maps.googleapis.com/maps/api/staticmap...,370000,3.0,-77.916370,34.233273,RECENTLY_SOLD,54310454,NaN,-1,3.0,1553.0,USA,USD,acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
912,8651,1970-01-01,SINGLE_FAMILY,0.429982,"418 Forest Hills Dr, Wilmington, NC 28403",https://maps.googleapis.com/maps/api/staticmap...,373500,4.0,-77.913490,34.232530,RECENTLY_SOLD,54310438,NaN,-1,3.0,2098.0,USA,USD,acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
'''Add value column'''
condition_list = [
    relevant_homes['Price/sqft'] >= 175,
    (relevant_homes['Price/sqft'] < 175) & (relevant_homes['Price/sqft'] >= 150),
    relevant_homes['Price/sqft'] < 150
]

choice_list = ['Expensive', 'Reasonable', 'Affordable']

relevant_homes['value'] = np.select(condition_list, choice_list)
relevant_homes

,Unnamed: 0,dateSold,propertyType,lotAreaValue,address,imgSrc,price,bedrooms,longitude,latitude,listingStatus,zpid,contingentListingType,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,newConstructionType,unit,listingSubType.is_newHome,listingSubType.is_FSBO,listingSubType.is_accepting_backup_offers,Price/sqft,value
0,13,NaN,SINGLE_FAMILY,2178.000000,"917 Chestnut Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/4560b29811f...,289000,3.0,-77.937355,34.238650,FOR_SALE,54301824,NaN,-1,2.0,1151.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,NaN,NaN,251.086012,Expensive
1,27,NaN,SINGLE_FAMILY,2613.600000,"707 N 11th Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/757e38dfb46...,275000,2.0,-77.935905,34.244090,FOR_SALE,54300701,NaN,-1,2.0,1149.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,NaN,NaN,239.338555,Expensive
2,33,NaN,SINGLE_FAMILY,4791.600000,"105 N 13th Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/0b515c83515...,239900,3.0,-77.933334,34.238210,FOR_SALE,54301987,NaN,-1,2.0,1463.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,NaN,NaN,163.978127,Reasonable
3,34,NaN,SINGLE_FAMILY,0.250000,"449 N 21st Street, Wilmington, NC 28405",https://photos.zillowstatic.com/fp/608ccf33669...,315000,3.0,-77.924840,34.244804,FOR_SALE,54300922,NaN,-1,2.0,1366.0,USA,USD,acres,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,40,NaN,SINGLE_FAMILY,1742.400000,"319 Anderson Street, Wilmington, NC 28401",https://photos.zillowstatic.com/fp/1f63ff91311...,149997,2.0,-77.938890,34.240288,FOR_SALE,54300588,NaN,-1,1.0,1038.0,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,NaN,NaN,144.505780,Affordable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,8635,1970-01-01,SINGLE_FAMILY,5662.000000,"605 S 3rd St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,375000,3.0,-77.944670,34.227585,RECENTLY_SOLD,54310682,NaN,-1,2.0,2132.0,USA,USD,sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175.891182,Expensive
911,8637,1970-01-01,SINGLE_FAMILY,3484.000000,"219 N 7th St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,380000,3.0,-77.941330,34.239124,RECENTLY_SOLD,54301614,NaN,-1,2.0,1821.0,USA,USD,sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.676551,Expensive
912,8651,1970-01-01,SINGLE_FAMILY,0.429982,"418 Forest Hills Dr, Wilmington, NC 28403",https://maps.googleapis.com/maps/api/staticmap...,373500,4.0,-77.913490,34.232530,RECENTLY_SOLD,54310438,NaN,-1,3.0,2098.0,USA,USD,acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
913,8653,1970-01-01,SINGLE_FAMILY,0.419995,"209 S 15th St, Wilmington, NC 28401",https://maps.googleapis.com/maps/api/staticmap...,380000,2.0,-77.929510,34.233814,RECENTLY_SOLD,54309388,NaN,-1,2.0,1568.0,USA,USD,acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [49]:
relevant_homes[['lotAreaValue', 'price', 'livingArea', 'Price/sqft']].describe()

,lotAreaValue,price,livingArea,Price/sqft
count,913.000000,915.000000,915.000000,845.000000
mean,2550.397686,231323.578142,1517.680874,169.720704
std,2903.876783,84232.120635,1346.961719,60.080438
min,0.000000,500.000000,1000.000000,0.437063
25%,0.120000,178000.000000,1139.000000,135.520685
50%,2178.000000,212000.000000,1300.000000,169.425511
75%,4791.000000,299450.000000,1582.000000,203.555556
max,10454.400000,695000.000000,26149.000000,413.486005


In [50]:
'''Set up Google Maps API'''

google_api_key = api_keys.loc[api_keys['API'] == 'google']['KEY'][1]
google_api_key
gmaps.configure(api_key=google_api_key)

In [51]:
'''Create Google Map of Wilmington, NC'''
wilmington_coordinates = (34.236509, -77.933831)
fig = gmaps.figure(center=wilmington_coordinates, zoom_level=14, map_type = 'ROADMAP')

In [52]:
'''Get house locations'''
houses = relevant_homes[['latitude', 'longitude', 'value', 'address', 'imgSrc', 'price','Price/sqft', 'listingStatus']]

info_box_template = """
<dl>
<dt>Address</dt><dd>{address}</dd>
<dt>Price</dt><dd>{price}</dd>
<dt>Price/sqft</dt><dd>{Price/sqft}</dd>
<dt>Status</dt><dd>{listingStatus}</dd>
<dt>Pictures</dt><dd><a href={imgSrc} target="_blank">Photos</a></dd>
</dl>
"""

affordable_houses = houses.loc[houses['value'] == 'Affordable']
affordable_location = affordable_houses[['latitude', 'longitude']]
affordable_house_info = [info_box_template.format(**house) for house in affordable_houses[['address', 'imgSrc', 'price','Price/sqft', 'listingStatus']].to_dict(orient='records')]

expensive_houses = houses.loc[houses['value'] == 'Expensive']
expensive = expensive_houses[['latitude', 'longitude']]
expensive_house_info = [info_box_template.format(**house) for house in expensive_houses[['address', 'imgSrc', 'price','Price/sqft', 'listingStatus']].to_dict(orient='records')]

reasonable_houses = houses.loc[houses['value'] == 'Reasonable']
reasonable = reasonable_houses[['latitude', 'longitude']]
reasonable_house_info = [info_box_template.format(**house) for house in reasonable_houses[['address', 'imgSrc', 'price','Price/sqft', 'listingStatus']].to_dict(orient='records')]

affordable_layer = gmaps.symbol_layer(affordable_location, fill_color="green", stroke_color="green", scale=6, info_box_content=affordable_house_info)
expensive_layer = gmaps.symbol_layer(expensive, fill_color='red', stroke_color='red', scale=6, info_box_content=expensive_house_info)
reasonable_layer = gmaps.symbol_layer(reasonable, fill_color='yellow', stroke_color='yellow', scale=6, info_box_content=reasonable_house_info)

fig.add_layer(affordable_layer)
fig.add_layer(expensive_layer)
fig.add_layer(reasonable_layer)
fig

Figure(layout=FigureLayout(height='420px'))